# Project 3 - Constructing a database using API and Webscrapping

# Importing libraries

In [56]:
# Required Liberaries:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
from tqdm.auto import tqdm
import sqlalchemy as db

# Functions

In [2]:
def get_response(url):
    response = requests.get(url, headers = headers)
    return response

# USING API

![Twitch Logo](https://theroguewolfe.files.wordpress.com/2014/08/twitchlogo.png)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Twitch** is a video live streaming service operated by Twitch Interactive, a subsidiary of Amazon. Introduced in June 2011 as a spin-off of the general-interest streaming platform, Justin.tv, the site primarily focuses on video game live streaming, including broadcasts of eSports competitions, in addition to music broadcasts, creative content, and more recently, "in real life" streams. Content on the site can be viewed either live or via video on demand.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;I decided to use **Twitch Api** to search for data in this project, because reading its documentation it seemed simple to get access and its practical endpoints to work with. And the access is really intuitive! But the data is limited to some endpoint options, I tried to prioritize some information that can be used later relating to other data.

## Creating tables

### Games, Viewers and Channels

This table will show the games being played the most and their respective channel numbers.

In [52]:
headers = {'Client-ID' : 'h8zvs5dtl6ukn16cvf8s2ade7xw6km', 'Accept' : 'application/vnd.twitchtv.v5+json'}

gameviewers = pd.DataFrame()
for i in range(0,11):
    response = get_response('https://api.twitch.tv/kraken/games/top?offset='+str(i) +'&Limit=100')
    games = response.json()['top']
    
    
    for game in games:
        today = datetime.date.today()
        hour_list = datetime.datetime.now().strftime("%H:%M:%S")
        game_id = game['game']['_id']
        game_name = game['game']['name']
        viewers = int(game['viewers'])
        channels = int(game['channels'])
        mini_df = pd.DataFrame({'date':[today],
                                'hour': [hour_list],
                                'id':[game_id],
                               'name': [game_name],
                               'viewers': [viewers],
                               'channels': [channels]})
        gameviewers = pd.concat([gameviewers,mini_df])

gameviewers = gameviewers.reset_index(drop=True)

In [53]:
gameviewers

,date,hour,id,name,viewers,channels
0,2020-03-30,21:08:01,509658,Just Chatting,321508,3161
1,2020-03-30,21:08:01,512710,Call of Duty: Modern Warfare,181898,7252
2,2020-03-30,21:08:01,21779,League of Legends,180176,5534
3,2020-03-30,21:08:01,32982,Grand Theft Auto V,125019,1803
4,2020-03-30,21:08:01,33214,Fortnite,119507,6834
...,...,...,...,...,...,...
101,2020-03-30,21:08:03,513143,Teamfight Tactics,25670,675
102,2020-03-30,21:08:03,491931,Escape From Tarkov,24435,1466
103,2020-03-30,21:08:03,490393,Motorsport Manager,23476,17
104,2020-03-30,21:08:03,497057,Destiny 2,21027,1000


### Most viewed channels of top games

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Using the endpoint of the previous table, we take the most watched games and search the channel endpoint to find out which channels are transmitting and receiving so many views.

In [54]:
stream_channel = pd.DataFrame()

for i in tqdm(range(10)):
    response = get_response('https://api.twitch.tv/kraken/games/top')
    
    games = response.json()
    top10games = (games['top'][i]['game']['name'])
    url_query = 'https://api.twitch.tv/kraken/search/streams?query='+top10games+'&Limit=100'

    response = requests.get(url_query, headers = headers)
    streams = response.json()['streams']
    
    for i in range(10):
        try:
            today = datetime.date.today()
            hour_list = datetime.datetime.now().strftime("%H:%M:%S")
            stream_id = streams[i]['channel']['_id']
            stream_name = streams[i]['channel']['name']
            stream_game = streams[i]['game']
            stream_viewers = int(streams[i]['viewers'])
            stream_lang = streams[i]['channel']['broadcaster_language']
            stream_followers = int(streams[i]['channel']['followers'])
            stream_views = int(streams[i]['channel']['views'])
            mini_df = pd.DataFrame({'date':[today],
                                    'hour':[hour_list],
                                    'streamer_id':[stream_id],
                                    'streamer_name': [stream_name],
                                    'game_name': [stream_game],
                                    'viewers': [stream_viewers],
                                    'language': [stream_lang],
                                    'followers': [stream_followers],
                                    'views': [stream_views]})
            stream_channel = pd.concat([stream_channel,mini_df])
        except:
            pass
stream_channel = stream_channel.reset_index(drop=True)

In [55]:
stream_channel = stream_channel.sort_values(by = 'viewers', ascending = False)
stream_channel.head(20).reset_index()

,index,date,hour,streamer_id,streamer_name,game_name,viewers,language,followers,views
0,10,2020-03-30,21:09:05,39276140,rubius,minecraft,128843,es,3709821,46018744
1,0,2020-03-30,21:09:04,17337557,drdisrespect,Call of Duty: Modern Warfare,37587,en,4157833,167976787
2,18,2020-03-30,21:09:05,51496027,loltyler1,League of Legends,25418,en,3173525,155354904
3,11,2020-03-30,21:09:05,26301881,sodapoppin,Just Chatting,22228,en,2695052,309164237
4,56,2020-03-30,21:09:08,26490481,summit1g,Counter-Strike: Global Offensive,21496,en,4397078,342411737
5,1,2020-03-30,21:09:04,15564828,nickmercs,Call of Duty: Modern Warfare,20812,en,3322830,94730085
6,57,2020-03-30,21:09:08,506221215,stewie2klrvo,Counter-Strike: Global Offensive,20312,en,5,0
7,19,2020-03-30,21:09:05,47071880,yoda,League of Legends,18211,pt,1619916,161574816
8,64,2020-03-30,21:09:09,121059319,moonmoon,Mount & Blade II: Bannerlord,17679,en,876935,69171118
9,65,2020-03-30,21:09:09,49140130,handiofiblood,Mount & Blade II: Bannerlord,17591,de,608221,18725803


# WEB SCRAPING

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;For Web Scraping I decide to search for data on **Steam** and **Metacrit**, they are sites that are reference in the area and contain information that may be useful in the future.

## Getting data from Steam

![Steam logo](http://icons.iconarchive.com/icons/martz90/circle/128/steam-icon.png)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Steam is a video game digital distribution service by Valve. It was launched as a standalone software client in September 2003 as a way for Valve to provide automatic updates for their games, and expanded to include games from third-party publishers. Steam has also expanded into an online web-based and mobile digital storefront. Steam offers digital rights management (DRM), matchmaking servers, video streaming, and social networking services. It also provides the user with installation and automatic updating of games, and community features such as friends lists and groups, cloud saving, and in-game voice and chat functionality.

### Getting the top 50 selled games on Steam

In [7]:
url = 'https://store.steampowered.com/search/?filter=topsellers'
html = requests.get(url).content
soup = BeautifulSoup(html)

mostselledgames = [games.text.strip().replace('\n','') for games in soup.find_all('div', attrs = {'class': 'col search_name ellipsis'})]

datereleased = [date.text for date in soup.find_all('div', attrs = {'class': 'col search_released responsive_secondrow'})]

try:
    discount = [discount.text.strip().replace('-','').replace('%','') for discount in soup.find_all('div', attrs = {'class': 'col search_discount responsive_secondrow'})]
except:
    discount = [discount.text.strip() for discount in soup.find_all('div', attrs = {'class': 'col search_discount responsive_secondrow'})]

try:
    price = [float(price.text.strip().split()[-1].replace(',','.')) for price in soup.find_all('div', attrs = {'class': 'col search_price_discount_combined responsive_secondrow'})]
except:
    price = [price.text.strip().split()[-1].replace(',','.') for price in soup.find_all('div', attrs = {'class': 'col search_price_discount_combined responsive_secondrow'})]
date_list = [datetime.date.today() for i in range(len(mostselledgames))]
data = list(zip(date_list, mostselledgames, datereleased, discount, price))
headers = ['date', 'game_name', 'date_released', 'discount', 'price R$']
mostselled_games = pd.DataFrame(np.array(data), columns = headers)

#### Most Selled Games DataFrame

In [8]:
mostselled_games

,date,game_name,date_released,discount,price R$
0,2020-03-30,Mount & Blade II: Bannerlord,"30 Mar, 2020",10,134.99
1,2020-03-30,Grand Theft Auto V,"13 Apr, 2015",50,34.99
2,2020-03-30,RESIDENT EVIL 3,2020/04/03,,129.99
3,2020-03-30,Monster Hunter World: Iceborne,"9 Jan, 2020",25,67.49
4,2020-03-30,Pummel Party,"20 Sep, 2018",34,19.13
5,2020-03-30,Metro Exodus - Gold Edition,,55,55.79
6,2020-03-30,The Witcher 3: Wild Hunt - Game of the Year Ed...,"30 Aug, 2016",70,29.99
7,2020-03-30,Euro Truck Simulator 2,"12 Oct, 2012",75,9.99
8,2020-03-30,Grand Theft Auto V: Premium Online Edition,,52,43.11
9,2020-03-30,MONSTER HUNTER: WORLD,"9 Aug, 2018",34,46.19


### Getting a view of players online on steam by game

In [9]:
url = 'https://store.steampowered.com/stats/Steam-Game-and-Player-Statistics'
html = requests.get(url).content
soup = BeautifulSoup(html)
players_online = [players.text.strip().split()[0] for players in soup.find_all('tr', attrs = {'class': 'player_count_row'})]
daily_peak = [players.text.strip().split()[1] for players in soup.find_all('tr', attrs = {'class': 'player_count_row'})]
game_name = [players.find('a').text for players in soup.find_all('tr', attrs = {'class': 'player_count_row'})]

date_list = [datetime.date.today() for i in range(len(game_name))]
hour_list = [datetime.datetime.now().strftime("%H:%M:%S") for i in range(len(game_name))]
data = list(zip(date_list, hour_list, game_name, daily_peak, players_online))
headers = ['date_list', 'hour_list', 'game_name', 'daily_peak', 'players_online']
players_online = pd.DataFrame(np.array(data), columns = headers)

#### Players Online by Games DataFrame

In [10]:
players_online.head(10)

,date_list,hour_list,game_name,daily_peak,players_online
0,2020-03-30,18:51:28,Counter-Strike: Global Offensive,"1,138,639","830,953"
1,2020-03-30,18:51:28,Dota 2,"705,206","432,816"
2,2020-03-30,18:51:28,Mount & Blade II: Bannerlord,"178,562","172,214"
3,2020-03-30,18:51:28,Football Manager 2020,"168,765","138,569"
4,2020-03-30,18:51:28,Tom Clancy's Rainbow Six Siege,"167,773","117,089"
5,2020-03-30,18:51:28,Grand Theft Auto V,"198,917","114,062"
6,2020-03-30,18:51:28,PLAYERUNKNOWN'S BATTLEGROUNDS,"530,038","87,054"
7,2020-03-30,18:51:28,Rocket League,"93,925","83,657"
8,2020-03-30,18:51:28,Rust,"91,891","80,215"
9,2020-03-30,18:51:28,Team Fortress 2,"82,207","76,793"


## Getting data from Metacritic

![Metacritic Logo](https://upload.wikimedia.org/wikipedia/commons/thumb/4/48/Metacritic_logo.svg/726px-Metacritic_logo.svg.png)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Metacritic** is a website that aggregates reviews of films, TV shows, music albums, video games and formerly, books. The site provides an excerpt from each review and hyperlinks to its source. A color of green, yellow or red summarizes the critics' recommendations. It is regarded as the foremost online review aggregation site for the video game industry.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Metacritic's** scoring converts each review into a percentage, either mathematically from the mark given, or what the site decides subjectively from a qualitative review. Before being averaged, the scores are weighted according to the critic's fame, stature, and volume of reviews. The website won two Webby Awards for excellence as an aggregation website. Criticism has focused on the assessment system, alleged third-party attempts to influence the scores, and the lack of staff oversight of user reviews.

### Creating a table of games and their notes

In [11]:
url = 'https://www.metacritic.com/browse/games/score/metascore/all/pc/filtered?page=1'

In [12]:
df = pd.DataFrame()
i = 0
status_code = 200

for i in range(2):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    url = f'https://www.metacritic.com/browse/games/score/metascore/all/pc/filtered?page={i}'
    html = requests.get(url, headers=headers).content
    soup = BeautifulSoup(html)
    game_name = [game.text.strip() for game in soup.find_all('div', attrs = {'class': 'basic_stat product_title'})]
    metacritic_score = [metascore.text for metascore in soup.find_all('div', attrs = {'class': 'metascore_w small game positive'})]
    user_score = [userscore.text.strip().replace('\n', '').split(':')[1] for userscore in soup.find_all('li', attrs = {'class': 'stat product_avguserscore'})]
    release_date = [release_date.text.strip().replace('\n', '') for release_date in soup.find_all('li', attrs = {'class': 'stat release_date full_release_date'})]
    
    data = list(zip(game_name, metacritic_score, user_score, release_date))
    headers = ['game_name', 'metacritic_score', 'user_score', 'release_date']
    metacrit = pd.DataFrame(np.array(data), columns=headers)

#### Games by Metacritic Score and Users Score

In [13]:
metacrit

,game_name,metacritic_score,user_score,release_date
0,Sid Meier's Civilization III,90,8.4,"Release Date:Oct 30, 2001"
1,Silent Hunter III,90,7.8,"Release Date:Mar 15, 2005"
2,Bayonetta,90,8.1,"Release Date:Apr 11, 2017"
3,Sid Meier's Civilization V,90,8.0,"Release Date:Sep 21, 2010"
4,Falcon 4.0: Allied Force,90,7.8,"Release Date:Jun 28, 2005"
...,...,...,...,...
95,Ori and the Blind Forest: Definitive Edition,88,8.8,"Release Date:Apr 27, 2016"
96,Shogo: Mobile Armor Division,88,8.4,"Release Date:Sep 30, 1998"
97,Thirty Flights of Loving,88,5.0,"Release Date:Aug 20, 2012"
98,Hearthstone: Heroes of Warcraft,88,5.9,"Release Date:Mar 11, 2014"


# CONNECTING PYTHON WITH POSTGRE

## Creating a function to connect and create a table

In [22]:
def create_table(title: str, df):
    engine = db.create_engine('postgresql://postgres:1fYS.9:f@localhost/games')
    conn = engine.connect()
    df.to_sql(title, con=conn, if_exists = 'replace', index=False)
    conn.close()

In [67]:
create_table(title = 'game_viewers', df = gameviewers)

In [68]:
create_table(title = 'top_channels', df = stream_channel)

In [69]:
create_table(title = 'steam_topselledgames', df = mostselled_games)

In [70]:
create_table(title = 'steam_playersonline', df = players_online)

In [71]:
create_table(title = 'metacrit_review', df = metacrit)

## Inserting New data to Tables

In [18]:
def update_table(title: str, df):
    engine = db.create_engine('postgresql://postgres:1fYS.9:f@localhost/games')
    conn = engine.connect()
    df.to_sql(title, con=conn, if_exists='append', index = False)
    conn.close()

In [31]:
update_table(title = 'game_viewers', df = gameviewers)

In [32]:
update_table(title = 'top_channels', df = stream_channel)

In [33]:
update_table(title = 'steam_topselledgames', df = mostselled_games)

In [34]:
update_table(title = 'steam_playersonline', df = players_online)